In [1]:
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt

# Lösung 1: Wie haben sich die Ausleihzahlen von EBOOKs im Vergleich zu BOOKs in den letzten zehn Jahren entwickelt?

## Step 1: Retrieve the datasets

In [ ]:
# wieviele zeilen pro jahr?
limit = 10000

# range(n, n-1) erzeugt eine sequenz von n bis n-1
# str() notwendig für concat von integer mit string, da datentypen angeglichen werden müssen
for i in range(2011,2021):
    url = "https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=checkoutyear=" + str(i) + "&$limit=" + str(limit)
    print("Retrieving: \t" + url)
    
    checkout_file = "checkout_" + str(i) + ".csv"
    print("Writing: \t" + checkout_file)
    
    urllib.request.urlretrieve(url, checkout_file)
    print("Done\n")

Retrieving: 	https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=checkoutyear=2011&$limit=10000
Writing: 	checkout_2011.csv
Done

Retrieving: 	https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=checkoutyear=2012&$limit=10000
Writing: 	checkout_2012.csv
Done

Retrieving: 	https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=checkoutyear=2013&$limit=10000
Writing: 	checkout_2013.csv
Done

Retrieving: 	https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=checkoutyear=2014&$limit=10000
Writing: 	checkout_2014.csv
Done

Retrieving: 	https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=checkoutyear=2015&$limit=10000
Writing: 	checkout_2015.csv
Done

Retrieving: 	https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=checkoutyear=2016&$limit=10000
Writing: 	checkout_2016.csv


## Step 2: Transform into dataframe

In [ ]:
# leeren Dataframe erzeugen
checkouts_df = pd.DataFrame()

# gehe über alle Datasets und vereine sie in ein großes Dataframe
for i in range(2011,2021):
    checkout_file = "checkout_" + str(i) + ".csv"
    
    # temporäres Dataframe aus Dataset erzeugen
    print("Generating dataframe for: \t" + checkout_file)
    temp_df = pd.read_csv(checkout_file)
    
    
    # jedes temporäre Dataframe wird an das Gesamt-Dataframe drangehängt
    print("Add " + str(len(temp_df)) + " rows to aggregated dataframe")
    checkouts_df = pd.concat([checkouts_df,temp_df])
    
    print("Done\n")
    
print("Aggregated Dataframe has " + str(len(checkouts_df)) + " rows")

In [ ]:
checkouts_df

## Step 3: Visualize

In [ ]:
checkouts_ebook_df = checkouts_df[checkouts_df["materialtype"] == "EBOOK"]
checkouts_ebook_final_df = checkouts_ebook_df.groupby(by="checkoutyear").sum()
checkouts_ebook_final_df["checkouts"].plot(kind="bar")

In [ ]:
checkouts_book_df = checkouts_df[checkouts_df["materialtype"] == "BOOK"]
checkouts_book_final_df = checkouts_book_df.groupby(by="checkoutyear").sum()
checkouts_book_final_df["checkouts"].plot(kind="bar")

## Extra-Step 4: Lets look at all materialtypes at once ;-)

In [ ]:
%matplotlib inline

# ermittle alle materialtypen
materialtype_list = checkouts_df["materialtype"].unique()

# gehe in schleife durch alle materialtypen
for materialtype in materialtype_list:
    # filtere aus Dataframe nur den aktuellen materialtyp und mach die selbe Magie wie oben
    checkouts_materialtype_df = checkouts_df[checkouts_df["materialtype"] == materialtype]
    checkouts_materialtype_final_df = checkouts_materialtype_df.groupby(by="checkoutyear").sum()
    checkouts_materialtype_final_df["checkouts"].plot(kind="bar")
    
    # zeige sofort den Plot inklusive Titel
    plt.title(materialtype)
    plt.show()

# Lösung 2: TOP-X Titel

In [ ]:
# Wie viele TOP?
top = 5
# Welche Materialtypen? -> Achtung, muss vollständigen Zeitraum abdecken, damit's funktioniert
materialtype_top_list = ["EBOOK", "BOOK", "AUDIOBOOK"]

# gehe durch alle vorgegebenen Materialtypen
for materialtype in materialtype_top_list:
    # gehe durch alle Jahre
    for year in range(2011,2021):
        print("TOP " + str(top) + " " + materialtype + "s aus " + str(year) + "\n")
        # wir brauchen nur die ersten x -> head()
        checkouts_temp_df = checkouts_df[(checkouts_df["checkoutyear"] == year) & (checkouts_df["materialtype"] == materialtype)].sort_values("checkouts", ascending=False).head(top)
        # liste die TOP-X Titel auf
        for i in range(0,top):
            print("\t- TOP " + str(i+1) + ": " + checkouts_temp_df["title"].iloc[i])
        print("\n")
        
        checkouts_temp_df.plot.barh(x="title", y="checkouts").invert_yaxis()
        plt.show()
    print("------------------------------------------------------------------\n")

# Lösung 3: Für wieviel % der Ausleihen in dieser Mediengruppe sind diese TOP-X-Titel verantwortlich?

In [ ]:
# diesmal die top 10
top = 10

# temporary sum for the checkout of the top-x titles
sum_topx_temp = 0

# gehe erneut durch alle vorgegebenen Materialtypen
for materialtype in materialtype_top_list:
    # gehe erneut durch alle Jahre
    for year in range(2011,2021):
        print("Prozentualer Anteil der TOP " + str(top) + " " + materialtype + "s aus " + str(year) + " an Gesamtausleihen in dieser Mediengruppe\n")
        # wir brauchen das gesamte set -> kein head()
        checkouts_temp_df = checkouts_df[(checkouts_df["checkoutyear"] == year) & (checkouts_df["materialtype"] == materialtype)].sort_values("checkouts", ascending=False)
        # liste die TOP-X Titel auf
        for i in range(0,top):
            # kumulierte ausleihen der top-x
            sum_topx_temp += checkouts_temp_df["checkouts"].iloc[i]
        # summe aller ausleihen der materialgruppe
        sum_checkouts_temp = checkouts_temp_df["checkouts"].sum()
        # prozentuales verhältnis zu gesamtausleihen
        #print(sum_topx_temp)
        #print(sum_checkouts_temp)
        percentage = (sum_topx_temp / sum_checkouts_temp) * 100
        #print(percentage)
        # reset sum_topx_temp
        sum_topx_temp = 0
        # mache Tortendiagram
        labels = 'Top ' + str(top) + ' ' + materialtype + 's', 'Restliche ' + materialtype + 's'
        sizes = [percentage.round(1), 100-percentage.round(1)]
        explode = (0.2, 0)
        fig, chart = plt.subplots()
        chart.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
        chart.axis('equal')
        plt.show()
    print("------------------------------------------------------------------\n")